In [1]:
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import pandas as pd

# from setting import parse_opts 
# from datasets.brains18 import BrainS18Dataset 
# from model import generate_model

import torch
import numpy as np
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import time
# from utils.logger import log
from scipy import ndimage

import warnings
warnings.filterwarnings(action='ignore')

import pathlib
import numpy as np
from scipy.interpolate import interp1d
import random

import cv2 
from torch.utils.data import DataLoader, Dataset
import albumentations as A

In [59]:
# 데이터 분석 최소 최대 값
# train = 

min = 0 
max = 0

x_val = 0
y_val = 0 
z_val = 0 

total = 0 

for idx, _ in train.iterrows():
    fname = train.iloc[idx, 0].split('/')[4]
    file_name = train.iloc[idx, 0].split('/')[5]
    # print(file_name)
    fid = open(os.path.join(data_path, fname, file_name), 'rb')

    img = np.fromfile(fid, dtype='int16', sep='')

    fid.close()
    
    if min > img.min():
        min = img.min() 
        
    if max < img.max():
        max = img.max()
        
    x, y, z = file_name.split('.')[1].split('x')
    
    x_val += int(x)
    y_val += int(y)
    z_val += int(z)
    
    total += (int(x) + int(y) + int(z))

print(f'min : {min} max : {max}')
print(f'x_val : {x_val} y_val : {y_val} z_val : {z_val} total : {total}')
print(f'AVG // x_val : {x_val / len(train)} y_val : {y_val / len(train)} z_val : {z_val / len(train)} total : {total / (len(train) * 3)}')


# print(img)
# test_img = np.reshape(img, [46, 46, 32])
# plt.imshow(test_img)
# train_images.append(img)
# test_img.shape, test_img.min(), test_img.max()


min : -3024 max : 3071
x_val : 24290 y_val : 24300 z_val : 12418 total : 61008
AVG // x_val : 50.604166666666664 y_val : 50.625 z_val : 25.870833333333334 total : 42.36666666666667


In [2]:
base_path = 'C:/Users/user/Desktop/datasets/Survival' 
# data_path = 'C:/Users/user/Desktop/Survival/DatasetNpz'
data_path = os.path.join(base_path, 'minraw')
# path = os.path.join(path, )

In [3]:
df = pd.read_excel(base_path + '/tabular_data_220531.xlsx')
df

,ID1,age,sex,diagnosis,stage,nodule.location,past.cancer.history,familyhistory,smoking,death,overall.survival,label.for.contrastive.learning,nodule.type,solidportion,exclusion,recurrence,recurrence.time,Unnamed: 17,Unnamed: 18,input variable candidates
0,10594052N1,68,F,AD,pT1a,LUL,1.0,0.0,0.0,0,66.3,0,Part-Solid,6,0,0.0,68.1,NaN,NaN,outcome variable (=label)
1,10798739N0,75,F,AD,pT1a,RLL,0.0,0.0,0.0,0,86.2,0,Solid,18,0,0.0,88.0,NaN,NaN,label for supervised contrastive learning
2,11043665N1,62,F,AD,pT2a,RLL,0.0,0.0,0.0,0,109.3,0,Solid,31,0,0.0,59.5,NaN,NaN,"input variable (CT 영상을 통해 학습할 수 있는 정보로 판단되나, 혹..."
3,11599315N0,66,M,AD,pT1a,LUL,1.0,0.0,1.0,0,74.6,0,Non-Solid,0,0,0.0,72.5,NaN,missing value 존재함. Imputation 필요.,past.cancer.history
4,12669039N0,76,F,AD,pT1a,RML,0.0,0.0,0.0,0,87.4,0,Non-Solid,0,0,0.0,86.5,NaN,missing value 존재함. Imputation 필요.,familyhistory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,44506128N2,69,M,AD,pT1a,RUL,0.0,0.0,1.0,1,58.4,1,Solid,18,0,0.0,57.1,NaN,NaN,NaN
884,45965117N2,62,M,AD,pT1a,LLL,0.0,0.0,1.0,0,79.8,0,Solid,19,0,0.0,58.1,NaN,NaN,NaN
885,22531016N0,65,M,AD,pT2a,LUL,0.0,0.0,0.0,0,97.7,0,Solid,47,0,0.0,95.8,NaN,NaN,NaN
886,43332843N0,83,F,AD,pT3,RLL,0.0,0.0,0.0,1,82.1,0,Solid,74,0,0.0,31.7,NaN,NaN,NaN


In [4]:
def getId(x):
    # print(x)
    return str(x[:-2])


df['ID2'] = df['ID1'].map(lambda x : str(getId(x)))

In [5]:
# df = df.astype({'ID2' : str})
df['ID2'] = df['ID2'].astype(np.int64)
df.info()
# df['']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID1                             888 non-null    object 
 1   age                             888 non-null    int64  
 2   sex                             888 non-null    object 
 3   diagnosis                       888 non-null    object 
 4   stage                           888 non-null    object 
 5   nodule.location                 888 non-null    object 
 6   past.cancer.history             873 non-null    float64
 7   familyhistory                   868 non-null    float64
 8   smoking                         885 non-null    float64
 9   death                           888 non-null    int64  
 10  overall.survival                888 non-null    float64
 11  label.for.contrastive.learning  888 non-null    int64  
 12  nodule.type                     888 

In [6]:
train_df = pd.read_csv(base_path + '/DenseTrain.csv')
# train_df

train_df = pd.merge(train_df, df, how='inner', left_on='1', right_on='ID2')
train_df

,0,1,ID1,age,sex,diagnosis,stage,nodule.location,past.cancer.history,familyhistory,...,label.for.contrastive.learning,nodule.type,solidportion,exclusion,recurrence,recurrence.time,Unnamed: 17,Unnamed: 18,input variable candidates,ID2
0,/home/user/minraw/42383475/NoduleRaw-000.46x46...,42383475,42383475N0,77,M,AD,pT2a,LUL,0.0,0.0,...,0,Solid,27,0,1.0,22.2,NaN,NaN,NaN,42383475
1,/home/user/minraw/35335632/NoduleRaw-000.24x24...,35335632,35335632N1,67,F,AD,pT1a,LUL,1.0,0.0,...,0,Solid,15,0,0.0,82.7,NaN,NaN,NaN,35335632
2,/home/user/minraw/32627008/NoduleRaw-000.36x34...,32627008,32627008N1,57,F,AD,pT2a,RML,0.0,0.0,...,0,Solid,20,0,0.0,100.3,NaN,NaN,NaN,32627008
3,/home/user/minraw/39089917/NoduleRaw-000.34x34...,39089917,39089917N1,58,F,AD,pT1a,LUL,1.0,NaN,...,0,Part-Solid,11,0,0.0,60.2,NaN,NaN,NaN,39089917
4,/home/user/minraw/20951258/NoduleRaw-000.40x40...,20951258,20951258N0,76,F,AD,pT1a,RLL,0.0,0.0,...,0,Part-Solid,5,0,0.0,93.6,NaN,NaN,NaN,20951258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,/home/user/minraw/12822993/NoduleRaw-000.40x40...,12822993,12822993N1,66,F,AD,pT2a,LLL,0.0,0.0,...,0,Solid,21,0,1.0,18.3,NaN,missing value 존재함. Imputation 필요.,smoking,12822993
278,/home/user/minraw/40920429/NoduleRaw-000.126x1...,40920429,40920429N0,69,F,AD,pT2b,LLL,0.0,0.0,...,0,Solid,32,0,0.0,71.0,NaN,NaN,NaN,40920429
279,/home/user/minraw/46348056/NoduleRaw-000.20x20...,46348056,46348056N2,58,M,AD,pT1a,LLL,0.0,0.0,...,0,Solid,10,0,0.0,71.5,NaN,NaN,NaN,46348056
280,/home/user/minraw/43615896/NoduleRaw-000.34x34...,43615896,43615896N0,71,F,AD,pT1a,RUL,1.0,0.0,...,0,Solid,15,0,0.0,46.8,NaN,NaN,NaN,43615896


In [7]:
test_df = pd.read_csv(base_path + '/DenseTest.csv')

test_df = pd.merge(test_df, df, how='inner', left_on='1', right_on='ID2')
test_df

,0,1,ID1,age,sex,diagnosis,stage,nodule.location,past.cancer.history,familyhistory,...,label.for.contrastive.learning,nodule.type,solidportion,exclusion,recurrence,recurrence.time,Unnamed: 17,Unnamed: 18,input variable candidates,ID2
0,/home/user/minraw/43954407/NoduleRaw-000.52x52...,43954407,43954407N2,71,M,AD,pT1b,RUL,0.0,0.0,...,0,Solid,31,0,0.0,60.2,NaN,NaN,NaN,43954407
1,/home/user/minraw/37256379/NoduleRaw-000.24x24...,37256379,37256379N0,55,M,AD,pT1a,RLL,0.0,1.0,...,0,Solid,15,0,0.0,108.9,NaN,NaN,NaN,37256379
2,/home/user/minraw/31553902/NoduleRaw-000.28x28...,31553902,31553902N0,55,F,AD,pT1a,RUL,0.0,0.0,...,0,Part-Solid,8,0,0.0,98.1,NaN,NaN,NaN,31553902
3,/home/user/minraw/41729706/NoduleRaw-000.24x26...,41729706,41729706N1,58,M,AD,pT1a,LUL,0.0,0.0,...,0,Part-Solid,6,0,0.0,59.7,NaN,NaN,NaN,41729706
4,/home/user/minraw/44968827/NoduleRaw-000.30x30...,44968827,44968827N0,67,M,AD,pT2a,RUL,0.0,0.0,...,0,Part-Solid,10,0,0.0,59.7,NaN,NaN,NaN,44968827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,/home/user/minraw/45965117/NoduleRaw-000.30x30...,45965117,45965117N2,62,M,AD,pT1a,LLL,0.0,0.0,...,0,Solid,19,0,0.0,58.1,NaN,NaN,NaN,45965117
95,/home/user/minraw/37798417/NoduleRaw-000.36x36...,37798417,37798417N1,69,F,AD,pT1a,RML,1.0,0.0,...,0,Part-Solid,11,0,0.0,84.7,NaN,NaN,NaN,37798417
96,/home/user/minraw/28841711/NoduleRaw-000.68x68...,28841711,28841711N0,76,M,AD,pT2a,RLL,0.0,0.0,...,0,Part-Solid,18,0,0.0,64.0,NaN,NaN,NaN,28841711
97,/home/user/minraw/44418872/NoduleRaw-000.28x30...,44418872,44418872N1,57,F,AD,pT1a,RUL,1.0,0.0,...,0,Part-Solid,9,0,1.0,43.4,NaN,NaN,NaN,44418872


In [8]:
tune_df = pd.read_csv(base_path + '/DenseTune.csv')
tune_df = pd.merge(tune_df, df, how='inner', left_on='1', right_on='ID2')
tune_df

,0,1,ID1,age,sex,diagnosis,stage,nodule.location,past.cancer.history,familyhistory,...,label.for.contrastive.learning,nodule.type,solidportion,exclusion,recurrence,recurrence.time,Unnamed: 17,Unnamed: 18,input variable candidates,ID2
0,/home/user/minraw/44874890/NoduleRaw-000.24x24...,44874890,44874890N2,56,M,AD,pT1a,LUL,0.0,0.0,...,0,Solid,11,0,0.0,61.2,NaN,NaN,NaN,44874890
1,/home/user/minraw/46295626/NoduleRaw-000.20x20...,46295626,46295626N1,42,F,AD,pT1a,RLL,1.0,0.0,...,0,Part-Solid,4,0,0.0,72.1,NaN,NaN,NaN,46295626
2,/home/user/minraw/46305491/NoduleRaw-000.28x28...,46305491,46305491N1,72,F,AD,pT1a,LUL,0.0,0.0,...,0,Part-Solid,3,0,0.0,70.3,NaN,NaN,NaN,46305491
3,/home/user/minraw/28871433/NoduleRaw-000.32x32...,28871433,28871433N2,78,F,AD,pT1a,RUL,0.0,0.0,...,0,Solid,17,0,0.0,59.5,NaN,NaN,NaN,28871433
4,/home/user/minraw/32505665/NoduleRaw-000.78x80...,32505665,32505665N1,74,F,AD,pT2a,LUL,0.0,1.0,...,0,Part-Solid,16,0,0.0,100.4,NaN,NaN,NaN,32505665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,/home/user/minraw/45645893/NoduleRaw-000.96x98...,45645893,45645893N1,58,F,AD,pT2a,RUL,0.0,0.0,...,0,Part-Solid,17,0,0.0,6.6,NaN,NaN,NaN,45645893
92,/home/user/minraw/40991111/NoduleRaw-000.26x24...,40991111,40991111N1,43,F,AD,pT1a,RLL,0.0,0.0,...,0,Part-Solid,3,0,0.0,113.1,NaN,NaN,NaN,40991111
93,/home/user/minraw/24157483/NoduleRaw-000.80x78...,24157483,24157483N0,73,F,AD,pT1b,LLL,1.0,NaN,...,0,Solid,24,0,0.0,62.8,NaN,NaN,NaN,24157483
94,/home/user/minraw/32249112/NoduleRaw-000.46x48...,32249112,32249112N1,60,F,AD,pT1b,RUL,1.0,0.0,...,0,Part-Solid,12,0,0.0,44.3,NaN,NaN,NaN,32249112


In [9]:
# min : -3024 max : 3071

InterSize = 42
degree = 3
InterFun = 'cubic'
NormalMin = -3024
NormalMax = 3024

NormalRange = NormalMax - NormalMin

# dataroot = './DeepBx2F' # .raw file folder
dataroot = data_path 
dataroot = pathlib.Path(dataroot)
Rawfiles = list(dataroot.glob('*/*'))
Rawfiles = [str(path) for path in Rawfiles if str(path).split('\\')[-1][:2] != '._'] # .raw filename read

# Rawfiles

In [10]:
def normalization_raw(rawfile):
    # print(rawfile)
    x, y, z = rawfile.split('\\')[-1].split('.')[1].split('x')
    x, y, z = int(x), int(y), int(z)
    
    # print(x)
    # print(y)
    # print(z)
    

    arr = np.fromfile(rawfile, dtype=np.int16)
    # arr = np.reshape(rawfile, (z[index], y[index], x[index]))    
    arr = np.reshape(arr, (z, y, x))

    # Eliminate empty spaces in 2-fold data
    n = 0
    while(n < z):
        if np.min(arr[n]) == np.max(arr[n]):
            arr = np.delete(arr,n,0)
            z -= 1
            n -= 1
        n += 1
    
    n = 0
    while(n < y):
        if np.min(arr[:,n,:]) == np.max(arr[:,n,:]):
            arr = np.delete(arr,n,1)
            y -= 1
            n -= 1
        n += 1
            
    n = 0
    while(n < x):
        if np.min(arr[:,:,n]) == np.max(arr[:,:,n]):
            arr = np.delete(arr,n,2)
            x -= 1
            n -= 1
        n += 1

    InterXarr = np.zeros((z,y,InterSize))
    InterYarr = np.zeros((z,InterSize,InterSize))
    Interarr = np.zeros((InterSize,InterSize,InterSize))

    IndexArr = np.arange(x)
    newArr = np.linspace(0,x - 1,InterSize)
    for iy in range(y):
        for iz in range(z):
            interF = interp1d(IndexArr,arr[iz,iy,:],kind=InterFun)
            InterXarr[iz,iy,:] = interF(newArr)
            
    IndexArr = np.arange(y)
    newArr = np.linspace(0,y - 1,InterSize)
    for ix in range(InterSize):
        for iz in range(z):
            interF = interp1d(IndexArr,InterXarr[iz,:,ix],kind=InterFun)
            InterYarr[iz,:,ix] = interF(newArr)

    IndexArr = np.arange(z)
    newArr = np.linspace(0,z - 1,InterSize)
    for iy in range(InterSize):
        for ix in range(InterSize):
            interF = interp1d(IndexArr,InterYarr[:,iy,ix],kind=InterFun)
            Interarr[:,iy,ix] = interF(newArr)
        
    Normal = np.clip((Interarr - NormalMin) / NormalRange,0,1)
    
    return Normal

In [11]:
train_images = []
train_labels = []

for idx, val in train_df.iterrows():
    foldername = train_df.loc[idx, '1']
    rawfile = [file for file in Rawfiles if str(foldername) in file]
    
    # print(rawfile)
    rawfile = 'C:\\Users\\user\\Desktop\\datasets\\Survival\\minraw\\42383475\\NoduleRaw-000.46x46x32.raw'
    img = normalization_raw(rawfile)
    # lbl = 
    lbl = 0
    if train_df.loc[idx, 'recurrence.time'] >= 33:
        lbl = 1
    
    # print(f'img shape : {img.shape}')
    
    train_images.append(img)
    train_labels.append(lbl)
    
train_images = np.reshape(train_images, [-1, InterSize, InterSize, InterSize])
train_labels = np.reshape(train_labels, [-1, 1])


train_images.shape, train_labels.shape

((282, 42, 42, 42), (282, 1))

In [12]:
test_images = []
test_labels = []

for idx, val in test_df.iterrows():
    foldername = test_df.loc[idx, '1']
    rawfile = [file for file in Rawfiles if str(foldername) in file]
    
    # print(rawfile)
    rawfile = 'C:\\Users\\user\\Desktop\\datasets\\Survival\\minraw\\42383475\\NoduleRaw-000.46x46x32.raw'
    img = normalization_raw(rawfile)
    # lbl = 
    lbl = 0
    if test_df.loc[idx, 'recurrence.time'] >= 33:
        lbl = 1
    
    # print(f'img shape : {img.shape}')
    
    test_images.append(img)
    test_labels.append(lbl)
    
test_images = np.reshape(test_images, [-1, InterSize, InterSize, InterSize])
test_labels = np.reshape(test_labels, [-1, 1])


test_images.shape, test_labels.shape

((99, 42, 42, 42), (99, 1))

In [13]:
tune_images = []
tune_labels = []

for idx, val in tune_df.iterrows():
    foldername = tune_df.loc[idx, '1']
    rawfile = [file for file in Rawfiles if str(foldername) in file]
    
    # print(rawfile)
    rawfile = 'C:\\Users\\user\\Desktop\\datasets\\Survival\\minraw\\42383475\\NoduleRaw-000.46x46x32.raw'
    img = normalization_raw(rawfile)
    # lbl = 
    lbl = 0
    if tune_df.loc[idx, 'recurrence.time'] >= 33:
        lbl = 1
    
    # print(f'img shape : {img.shape}')
    
    tune_images.append(img)
    tune_labels.append(lbl)
    
tune_images = np.reshape(tune_images, [-1, InterSize, InterSize, InterSize])
tune_labels = np.reshape(tune_labels, [-1, 1])


tune_images.shape, tune_labels.shape

((96, 42, 42, 42), (96, 1))

# Learning Supervised Contrative Learning

In [16]:
# import cv2 
# from torch.utils.data import DataLoader, Dataset
# import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm
from collections import defaultdict
from torch.cuda import amp
from pytorch_metric_learning import losses
import timm
import copy

In [17]:
class CFG:
    seed = 42
    model_name = 'tf_efficientnet_b4_ns'
    # img_size = 512
    img_size = 42 
    scheduler = 'CosineAnnealingLR'
    T_max = 10
    lr = 1e-5
    min_lr = 1e-6
    batch_size = 16
    weight_decay = 1e-6
    num_epochs = 10
    num_classes = 11014
    # embedding_size = 512
    embedding_size = 42
    n_fold = 5
    n_accumulate = 4
    temperature = 0.1
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    

set_seed(CFG.seed)

In [19]:
class ShopeeDataset(Dataset):
    def __init__(self, images, labels, transforms=None):
        # self.root_dir = root_dir
        # self.df = df
        self.transforms = transforms
        self.images = images
        self.labels = labels
        
    def __len__(self):
        # return len(self.df)
        return len(self.images)
    
    def __getitem__(self, index):
        # img_path = self.df.iloc[index, -1]
        # img = cv2.imread(img_path)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # label = self.df.iloc[index, -3]
        
        # if self.transforms:
        #     img = self.transforms(image=img)["image"]
        
        img = self.images[index]
        label = self.labels[index]
            
        return img, label

In [20]:
data_transforms = {
    "train": A.Compose([
        # A.Resize(CFG.img_size, CFG.img_size),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        # A.Resize(CFG.img_size, CFG.img_size),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

In [21]:
def train_model(model, criterion, optimizer, scheduler, num_epochs, dataloaders, dataset_sizes, device, fold):
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    scaler = amp.GradScaler()

    for step, epoch in enumerate(range(1,num_epochs+1)):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train','valid']:
            if(phase == 'train'):
                model.train() # Set model to training mode
            else:
                model.eval() # Set model to evaluation mode
            
            running_loss = 0.0
            
            # Iterate over data
            # for inputs,labels in tqdm(dataloaders[phase]):
            for inputs,labels in dataloaders[phase]:
                inputs = inputs.to(CFG.device)
                labels = labels.to(CFG.device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    with amp.autocast(enabled=True):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        loss = loss / CFG.n_accumulate
                    
                    # backward only if in training phase
                    if phase == 'train':
                        scaler.scale(loss).backward()

                    # optimize only if in training phase
                    if phase == 'train' and (step + 1) % CFG.n_accumulate == 0:
                        scaler.step(optimizer)
                        scaler.update()
                        scheduler.step()
                        
                        # zero the parameter gradients
                        optimizer.zero_grad()


                running_loss += loss.item()*inputs.size(0)
            
            epoch_loss = running_loss/dataset_sizes[phase]            
            history[phase + ' loss'].append(epoch_loss)

            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))
            
            # deep copy the model
            if phase=='valid' and epoch_loss <= best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                PATH = f"Fold{fold}_{best_loss}_epoch_{epoch}.bin"
                torch.save(model.state_dict(), PATH)

        print()

    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss ",best_loss)

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

In [22]:
def run_fold(model, criterion, optimizer, scheduler, device, fold, num_epochs=10):
    # valid_df = df_train[df_train.fold == fold]
    # train_df = df_train[df_train.fold != fold]
    
    train_data = ShopeeDataset(train_images, train_labels, transforms=data_transforms["train"])
    valid_data = ShopeeDataset(test_images, test_labels, transforms=data_transforms["valid"])
    
    dataset_sizes = {
        'train' : len(train_data),
        'valid' : len(valid_data)
    }
    
    train_loader = DataLoader(dataset=train_data, batch_size=CFG.batch_size, num_workers=4, pin_memory=True, shuffle=True)
    valid_loader = DataLoader(dataset=valid_data, batch_size=CFG.batch_size, num_workers=4, pin_memory=True, shuffle=False)
    
    dataloaders = {
        'train' : train_loader,
        'valid' : valid_loader
    }

    model, history = train_model(model, criterion, optimizer, scheduler, num_epochs, dataloaders, dataset_sizes, device, fold)
    
    return model, history

In [23]:
model = timm.create_model(CFG.model_name, pretrained=True)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, CFG.embedding_size)


RuntimeError: Given groups=1, weight of size [48, 3, 3, 3], expected input[1, 42, 43, 43] to have 3 channels, but got 42 channels instead

In [24]:
class Model_3d_2_2d(nn.Module):
    def __init__(self, model , input_channels = 3):
        super().__init__()
        self.model = model
        self.cnn3d = nn.Conv3d( input_channels , 1 , 3)
        self.pool = nn.AdaptiveAvgPool3d((144))
        self.cnn2d = nn.Conv2d(144 , 3 , 3 , stride = 1 , padding = 1)
    
    def forward(self , x):
        x = self.cnn3d(x)
        x = x.squeeze(1)
        x = self.pool(x) #changing Depth , width , height to 144
        x = self.cnn2d(x)
        x = self.model(x)
        
        return x

In [ ]:

out = model(torch.randn(1, 3, CFG.img_size, CFG.img_size))
print(f'Embedding shape: {out.shape}')

model.to(CFG.device);

In [38]:
class SupervisedContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.1):
        super(SupervisedContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, feature_vectors, labels):
        # Normalize feature vectors
        feature_vectors_normalized = F.normalize(feature_vectors, p=2, dim=1)
        # Compute logits
        logits = torch.div(
            torch.matmul(
                feature_vectors_normalized, torch.transpose(feature_vectors_normalized, 0, 1)
            ),
            self.temperature,
        )
        return losses.NTXentLoss(temperature=0.07)(logits, torch.squeeze(labels))

In [39]:
criterion = SupervisedContrastiveLoss(temperature=CFG.temperature).to(CFG.device) # Custom Implementation
# criterion = losses.SupConLoss(temperature=CFG.temperature).to(CFG.device)
optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr)

In [40]:
model, history = run_fold(model, criterion, optimizer, scheduler, device=CFG.device, fold=0, num_epochs=CFG.num_epochs)

Epoch 1/10
----------


In [85]:
def normalize(volume):
    """Normalize the volume"""
    min = -1024
    max = 1396
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 4 
    desired_width = 24
    desired_height = 24
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

def process_scan(img):
    """Read and resize volume"""
    # Read scan
    # volume = read_nifti_file(path)
    # Normalize
    img = normalize(img)
    # Resize width, height and depth
    img = resize_volume(img)
    return img


train_images = np.array([process_scan(img) for img in train_images])

IndexError: tuple index out of range

In [52]:
train_images = np.reshape(train_images, [-1, 46, 46, 46])

ValueError: cannot reshape array of size 32501760 into shape (46,46,46)

In [7]:
model_path = './MedicalNet_pytorch_files2/pretrain/resnet_50.pth'

In [91]:
'''
Training code for MRBrainS18 datasets segmentation
Written by Whalechen
'''

from setting import parse_opts 
# from datasets.brains18 import BrainS18Dataset 
from model import generate_model
import torch
import numpy as np
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import time
from utils.logger import log
from scipy import ndimage
import os



In [ ]:

def train(data_loader, model, optimizer, scheduler, total_epochs, save_interval, save_folder, sets):
    # settings
    batches_per_epoch = len(data_loader)
    log.info('{} epochs in total, {} batches per epoch'.format(total_epochs, batches_per_epoch))
    loss_seg = nn.CrossEntropyLoss(ignore_index=-1)

    print("Current setting is:")
    print(sets)
    print("\n\n")     
    if not sets.no_cuda:
        loss_seg = loss_seg.cuda()
        
    model.train()
    train_time_sp = time.time()
    for epoch in range(total_epochs):
        log.info('Start epoch {}'.format(epoch))
        
        scheduler.step()
        log.info('lr = {}'.format(scheduler.get_lr()))
        
        for batch_id, batch_data in enumerate(data_loader):
            # getting data batch
            batch_id_sp = epoch * batches_per_epoch
            volumes, label_masks = batch_data

            if not sets.no_cuda: 
                volumes = volumes.cuda()

            optimizer.zero_grad()
            out_masks = model(volumes)
            # resize label
            [n, _, d, h, w] = out_masks.shape
            new_label_masks = np.zeros([n, d, h, w])
            for label_id in range(n):
                label_mask = label_masks[label_id]
                [ori_c, ori_d, ori_h, ori_w] = label_mask.shape 
                label_mask = np.reshape(label_mask, [ori_d, ori_h, ori_w])
                scale = [d*1.0/ori_d, h*1.0/ori_h, w*1.0/ori_w]
                label_mask = ndimage.interpolation.zoom(label_mask, scale, order=0)
                new_label_masks[label_id] = label_mask

            new_label_masks = torch.tensor(new_label_masks).to(torch.int64)
            if not sets.no_cuda:
                new_label_masks = new_label_masks.cuda()

            # calculating loss
            loss_value_seg = loss_seg(out_masks, new_label_masks)
            loss = loss_value_seg
            loss.backward()                
            optimizer.step()

            avg_batch_time = (time.time() - train_time_sp) / (1 + batch_id_sp)
            log.info(
                    'Batch: {}-{} ({}), loss = {:.3f}, loss_seg = {:.3f}, avg_batch_time = {:.3f}'\
                    .format(epoch, batch_id, batch_id_sp, loss.item(), loss_value_seg.item(), avg_batch_time))
          
            if not sets.ci_test:
                # save model
                if batch_id == 0 and batch_id_sp != 0 and batch_id_sp % save_interval == 0:
                #if batch_id_sp != 0 and batch_id_sp % save_interval == 0:
                    model_save_path = '{}_epoch_{}_batch_{}.pth.tar'.format(save_folder, epoch, batch_id)
                    model_save_dir = os.path.dirname(model_save_path)
                    if not os.path.exists(model_save_dir):
                        os.makedirs(model_save_dir)
                    
                    log.info('Save checkpoints: epoch = {}, batch_id = {}'.format(epoch, batch_id)) 
                    torch.save({
                                'ecpoch': epoch,
                                'batch_id': batch_id,
                                'state_dict': model.state_dict(),
                                'optimizer': optimizer.state_dict()},
                                model_save_path)
                            
    print('Finished training')            
    if sets.ci_test:
        exit()

In [31]:
# sets = parse_opts()   
sets = dict()
sets['ci_test']= False
sets['img_list'] = './toy_data/test_ci.txt' 
sets['n_epochs'] = 1
sets['no_cuda']= True
sets['data_root'] = data_path
sets['pretrain_path'] = model_path
sets['num_workers'] = 0
sets['model_depth'] = 10
sets['resnet_shortcut'] = 'A'
sets['input_D']= 64
sets['input_H']= 64
sets['input_W']= 64
sets['manual_seed']= 1
sets['model'] = 'resnet'
sets['n_seg_classes'] = 2
sets['phase'] = 'train'
sets['new_layer_names'] = ['conv_seg']
sets['learning_rate'] = 0.001
sets['gpu_id'] = 0
sets['resume_path'] = ''

In [32]:
from models import resnet


def create_model(opt):
    assert opt['model'] in [
        'resnet'
    ]

    # if opt['model'] == resnet:
    #     assert opt['model_depth'] in [10, 18, 34, 50, 101, 152, 200]
        
        # if opt['model_depth'] == 10:
    model = resnet.resnet10(
        sample_input_W=opt['input_W'],
        sample_input_H=opt['input_H'],
        sample_input_D=opt['input_D'],
        shortcut_type=opt['resnet_shortcut'],
        no_cuda=opt['no_cuda'],
        num_seg_classes=opt['n_seg_classes'])
    
    if opt['model_depth'] == 18:
        model = resnet.resnet18(
            sample_input_W=opt['input_W'],
            sample_input_H=opt['input_H'],
            sample_input_D=opt['input_D'],
            shortcut_type=opt['resnet_shortcut'],
            no_cuda=opt['no_cuda'],
            num_seg_classes=opt['n_seg_classes'])
    elif opt['model_depth'] == 34:
        model = resnet.resnet34(
            sample_input_W=opt['input_W'],
            sample_input_H=opt['input_H'],
            sample_input_D=opt['input_D'],
            shortcut_type=opt['resnet_shortcut'],
            no_cuda=opt['no_cuda'],
            num_seg_classes=opt['n_seg_classes'])
    elif opt['model_depth'] == 50:
        model = resnet.resnet50(
            sample_input_W=opt['input_W'],
            sample_input_H=opt['input_H'],
            sample_input_D=opt['input_D'],
            shortcut_type=opt['resnet_shortcut'],
            no_cuda=opt['no_cuda'],
            num_seg_classes=opt['n_seg_classes'])
    elif opt['model_depth'] == 101:
        model = resnet.resnet101(
            sample_input_W=opt['input_W'],
            sample_input_H=opt['input_H'],
            sample_input_D=opt['input_D'],
            shortcut_type=opt['resnet_shortcut'],
            no_cuda=opt['no_cuda'],
            num_seg_classes=opt['n_seg_classes'])
    elif opt['model_depth'] == 152:
        model = resnet.resnet152(
            sample_input_W=opt['input_W'],
            sample_input_H=opt['input_H'],
            sample_input_D=opt['input_D'],
            shortcut_type=opt['resnet_shortcut'],
            no_cuda=opt['no_cuda'],
            num_seg_classes=opt['n_seg_classes'])
    elif opt['model_depth'] == 200:
        model = resnet.resnet200(
            sample_input_W=opt['input_W'],
            sample_input_H=opt['input_H'],
            sample_input_D=opt['input_D'],
            shortcut_type=opt['resnet_shortcut'],
            no_cuda=opt['no_cuda'],
            num_seg_classes=opt['n_seg_classes'])
            
    
    if not opt['no_cuda']:
        
        if len(opt['gpu_id']) > 1:
            model = model.cuda() 
            model = nn.DataParallel(model, device_ids=opt['gpu_id'])
            net_dict = model.state_dict() 
        else:
            os.environ["CUDA_VISIBLE_DEVICES"]=str(opt['gpu_id'][0])
            model = model.cuda() 
            model = nn.DataParallel(model, device_ids=None)
            net_dict = model.state_dict()
    else:
        net_dict = model.state_dict()
    
    # load pretrain
    if opt['phase'] != 'test' and opt['pretrain_path']:
        print ('loading pretrained model {}'.format(opt['pretrain_path']))
        pretrain = torch.load(opt['pretrain_path'])
        pretrain_dict = {k: v for k, v in pretrain['state_dict'].items() if k in net_dict.keys()}
         
        net_dict.update(pretrain_dict)
        model.load_state_dict(net_dict)

        new_parameters = [] 
        for pname, p in model.named_parameters():
            for layer_name in opt['new_layer_names']:
                if pname.find(layer_name) >= 0:
                    new_parameters.append(p)
                    break

        new_parameters_id = list(map(id, new_parameters))
        base_parameters = list(filter(lambda p: id(p) not in new_parameters_id, model.parameters()))
        parameters = {'base_parameters': base_parameters, 
                      'new_parameters': new_parameters}

        return model, parameters

    return model, model.parameters()


In [34]:
# getting model
torch.manual_seed(sets['manual_seed'])
model, parameters = create_model(sets) 
# print (model)
# optimizer
if sets['ci_test']:
    params = [{'params': parameters
               , 'lr': sets['learning_rate']}]
else:
    params = [{ 'params': parameters['base_parameters'], 
               'lr': sets['learning_rate'] }, 
              { 'params': parameters['new_parameters'], 
               'lr': sets['learning_rate']*100 }]
    
# for p in params:
#     print(p)
#     print(f'type : {type(p)}, {type(p["lr"])}')
    
optimizer = torch.optim.SGD(params, momentum=0.9, weight_decay=1e-3)   
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

# train from resume
if sets['resume_path']:
    if os.path.isfile(sets['resume_path']):
        print("=> loading checkpoint '{}'".format(sets['resume_path']))
        checkpoint = torch.load(sets['resume_path'])
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
            .format(sets['resume_path'], checkpoint['epoch']))

# getting data
sets['phase'] = 'train'
if sets['no_cuda']:
    sets['pin_memory'] = False
else:
    sets['pin_memory'] = True    
    


loading pretrained model ./MedicalNet_pytorch_files2/pretrain/resnet_50.pth


In [64]:
from torch.utils.data import Dataset, DataLoader

class CTImageDataset(Dataset):
    def __init__(self, path): 
        self.path = path 
        self.images = np.load(os.path.join(data_path, path))
        self.files = np.load(os.path.join(data_path, path)).files
        
    def __getitem__(self, index): 
        idx = self.files[index]
        x = self.images[idx]
        x = torch.from_numpy(x).float()
        return x 
    
    def __len__(self):
        return len(self.files)

In [66]:
ct_image_dataset = CTImageDataset('Trainset.npz')

for i in range(len(ct_image_dataset)):
    print(ct_image_dataset[i].shape)

torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])
torch.Size([46, 46, 46])


In [ ]:
# training_dataset = BrainS18Dataset(sets['data_root'], sets['img_list'], sets)
# data_loader = DataLoader(training_dataset, batch_size=sets['batch_size'], shuffle=True, num_workers=sets['num_workers'], pin_memory=sets['pin_memory'])
from torchvision import transforms, datasets


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

image_datasets = {x: datasets.ImageFolder("../data/hymenoptera_data", data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = BATCH_SIZE, num_workers = 0, shuffle = True) for x in ['train', 'val']}



In [ ]:
# training
train(data_loader, model, optimizer, scheduler, total_epochs=sets['n_epochs'], save_interval=sets['save_intervals'], save_folder=sets['save_folder'], sets=sets) 